<a href="https://colab.research.google.com/github/taras-musakovskyi/colab-jupyter-fish-models/blob/main/Prompt_Conditioned_Underwater_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
Stable Diffusion 1.5 LoRA Training for Underwater Effects
Dataset: 1,639 underwater/flooded apartment images (3 sources with repeat weights)
Training: Google Colab A100 40GB with diffusers library
Models: Periodic upload to Google Drive
"""

# ============================================================================
# CELL 1: Environment Setup & Installation
# ============================================================================
!pip install -q diffusers transformers accelerate bitsandbytes safetensors xformers
!pip install -q torch torchvision --upgrade
!pip install -q peft huggingface_hub

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import json
import zipfile
import shutil
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm
import numpy as np
from dataclasses import dataclass
from typing import Optional, List
import time
import gc

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 21.5 MB/s eta 0:00:00
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
VRAM: 39.56 GB


In [ ]:
# ============================================================================
# CELL 2: Google Drive Authentication (for uploading results only)
# ============================================================================
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
import io

print("🔐 Authenticating with Google Drive for results upload...")

# Use Colab's built-in authentication
auth.authenticate_user()
from google.auth import default
creds, _ = default()

# Build Drive service
drive_service = build('drive', 'v3', credentials=creds)

print("✓ Authentication successful!")

# Set your results upload folder ID
GDRIVE_RESULTS_FOLDER_ID = "1N16Q01oVuaXLXrhGm4iB_anhzlK-I6El" #Prompt Conditioned UnderWater LoRA

if "YOUR_" in GDRIVE_RESULTS_FOLDER_ID or "_HERE" in GDRIVE_RESULTS_FOLDER_ID:
    print("\n⚠️  WARNING: Set GDRIVE_RESULTS_FOLDER_ID to upload results!")
else:
    print(f"✓ Results will be uploaded to folder ID: {GDRIVE_RESULTS_FOLDER_ID}")

🔐 Authenticating with Google Drive for results upload...
✓ Authentication successful!
✓ Results will be uploaded to folder ID: 1N16Q01oVuaXLXrhGm4iB_anhzlK-I6El


In [ ]:
# ============================================================================
# CELL 3: Training Configuration
# ============================================================================
@dataclass
class TrainingConfig:
    # Model settings
    pretrained_model_name: str = "runwayml/stable-diffusion-v1-5"
    resolution: int = 512

    # LoRA settings
    lora_rank: int = 32
    lora_alpha: int = 64
    lora_dropout: float = 0.05

    # Training settings
    num_epochs: int = 40
    batch_size: int = 8
    gradient_accumulation_steps: int = 1
    learning_rate: float = 8e-5
    lr_scheduler: str = "cosine"
    lr_warmup_steps: int = 100
    mixed_precision: str = "fp16"

    # Validation settings
    validation_every_n_epochs: int = 5
    validation_prompts: List[str] = None
    num_validation_images: int = 4

    # Checkpointing
    save_every_n_epochs: int = 10
    upload_after_epoch: int = 20
    upload_interval_epochs: int = 5

    # Paths
    output_dir: str = "/content/underwater_lora_output"
    dataset_dir: str = "/content/dataset"

    # Dataset repeat weights (for effective exposure balancing)
    repeat_weights: dict = None

    def __post_init__(self):
        if self.validation_prompts is None:
            self.validation_prompts = [
                "underwater apartment with caustic light patterns",
                "room transitioning to underwater",
                "apartment filled with water, bubbles floating",
                "clear underwater environment",
                "underwater caustic patterns dancing"

#                "underwater living room with light caustics on walls, blue tinted lighting, bubbles floating",
#                "flooded office with desk and chair, underwater atmosphere, water effects",
#                "apartment bedroom underwater with blue lighting and water ripples",
#                "kitchen with cabinets submerged in water, caustic light patterns, aquatic environment"
            ]

        if self.repeat_weights is None:
            # Shutterstock: 79%, Tank: 16%, Unsplash: 5%
            self.repeat_weights = {
                'shutterstock-augmented': 8,
                'my-tank-underwater-scenes-augmented': 1,
                'unsplash-augmented': 1
            }

config = TrainingConfig()

print("="*60)
print("UNDERWATER LORA TRAINING CONFIGURATION")
print("="*60)
print(f"Model: {config.pretrained_model_name}")
print(f"Resolution: {config.resolution}x{config.resolution}")
print(f"LoRA Rank: {config.lora_rank}")
print(f"Epochs: {config.num_epochs}")
print(f"Batch Size: {config.batch_size}")
print(f"Gradient Accumulation: {config.gradient_accumulation_steps}")
print(f"Effective Batch Size: {config.batch_size * config.gradient_accumulation_steps}")
print(f"Learning Rate: {config.learning_rate}")
print(f"\nRepeat Weights (for exposure balancing):")
for folder, weight in config.repeat_weights.items():
    print(f"  {folder}: {weight}x")
print("="*60)

UNDERWATER LORA TRAINING CONFIGURATION
Model: runwayml/stable-diffusion-v1-5
Resolution: 512x512
LoRA Rank: 32
Epochs: 40
Batch Size: 8
Gradient Accumulation: 1
Effective Batch Size: 8
Learning Rate: 8e-05

Repeat Weights (for exposure balancing):
  shutterstock-augmented: 8x
  my-tank-underwater-scenes-augmented: 1x
  unsplash-augmented: 1x


In [ ]:
# ============================================================================
# CELL 4: Upload Dataset from Local Machine
# ============================================================================
from google.colab import files

print("📦 Upload your dataset ZIP file (containing 3 augmented folders)")
print("   Expected structure inside ZIP:")
print("   - shutterstock-augmented/")
print("   - my-tank-underwater-scenes-augmented/")
print("   - unsplash-augmented/")
print()

uploaded = files.upload()

# Get the uploaded filename
zip_filename = list(uploaded.keys())[0]
print(f"\n✓ Uploaded: {zip_filename}")

# Extract dataset
print("📂 Extracting dataset...")
os.makedirs(config.dataset_dir, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(config.dataset_dir)

print("✓ Dataset extracted!")

# Verify folder structure
dataset_folders = []
for folder_name in config.repeat_weights.keys():
    folder_path = os.path.join(config.dataset_dir, folder_name)
    if os.path.exists(folder_path):
        num_images = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
        dataset_folders.append((folder_name, num_images))
        print(f"  ✓ {folder_name}: {num_images} images")
    else:
        print(f"  ⚠️  {folder_name}: NOT FOUND")

total_images = sum(count for _, count in dataset_folders)
print(f"\n📊 Total raw images: {total_images}")

# Calculate effective exposures
effective_exposures = sum(count * config.repeat_weights.get(name, 1)
                         for name, count in dataset_folders)
print(f"📊 Effective training exposures (with repeats): {effective_exposures}")

📦 Upload your dataset ZIP file (containing 3 augmented folders)
   Expected structure inside ZIP:
   - shutterstock-augmented/
   - my-tank-underwater-scenes-augmented/
   - unsplash-augmented/



Saving underwater-lora-dataset.zip to underwater-lora-dataset.zip

✓ Uploaded: underwater-lora-dataset.zip
📂 Extracting dataset...
✓ Dataset extracted!
  ✓ shutterstock-augmented: 531 images
  ✓ my-tank-underwater-scenes-augmented: 848 images
  ✓ unsplash-augmented: 250 images

📊 Total raw images: 1629
📊 Effective training exposures (with repeats): 5346


In [ ]:
# ============================================================================
# CELL 5: Custom Dataset with Prompt Conditioning
# ============================================================================
class UnderwaterDataset(Dataset):
    """
    Dataset with category-specific prompts for underwater LoRA training.
    """
    def __init__(self, dataset_dir, repeat_weights, resolution=512, tokenizer=None):
        self.resolution = resolution
        self.tokenizer = tokenizer
        self.transform = transforms.Compose([
            transforms.Resize(resolution, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(resolution),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])

        # Enhanced prompts per folder
        self.folder_prompts = {
            'shutterstock-augmented': [
                "underwater apartment with caustic light patterns",
                "underwater living room, blue tinted lighting",
                "submerged apartment room with water effects",
                "underwater room with light caustics on walls",
                "apartment filled with water, bubbles floating",
                "apartment becoming underwater",
                "room transitioning to underwater",
                "interior space with water filling in",
                "living space with underwater transformation",
                "room submerged in clear water"
            ],
            'my-tank-underwater-scenes-augmented': [
                "clear underwater environment",
                "pristine aquatic scene",
                "underwater space with visibility",
                "crystal clear underwater",
                "underwater with light rays"
            ],
            'unsplash-augmented': [
                "underwater caustic patterns dancing",
                "water refraction and light rays",
                "thick underwater atmosphere with particles",
                "underwater scene with volumetric lighting",
                "deep underwater environment"
            ]
        }

        # Collect paths WITH folder info
        self.image_data = []  # (path, folder_name) tuples

        for folder_name, repeat_count in repeat_weights.items():
            folder_path = os.path.join(dataset_dir, folder_name)

            if not os.path.exists(folder_path):
                print(f"⚠️  Skipping {folder_name} - folder not found")
                continue

            image_files = [
                os.path.join(folder_path, f)
                for f in os.listdir(folder_path)
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ]

            # Add each image repeat_count times with folder info
            for _ in range(repeat_count):
                for img_path in image_files:
                    self.image_data.append((img_path, folder_name))

            print(f"  {folder_name}: {len(image_files)} images × {repeat_count} repeats = {len(image_files) * repeat_count} effective")

        print(f"\n✓ Dataset ready: {len(self.image_data)} total training samples with prompts")

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, idx):
        image_path, folder_name = self.image_data[idx]

        # Load image
        image = Image.open(image_path).convert('RGB')

        # Apply transforms
        pixel_values = self.transform(image)

        # Random prompt from folder's set
        import random
        prompts = self.folder_prompts.get(folder_name, ["underwater scene"])
        prompt = random.choice(prompts)

        # Tokenize prompt for training
        input_ids = self.tokenizer(
            prompt,
            padding="max_length",
            truncation=True,
            max_length=77,
            return_tensors="pt"
        ).input_ids[0]

        return {
            'pixel_values': pixel_values,
            'input_ids': input_ids
        }

# Create dataset
print("="*60)
print("CREATING DATASET WITH PROMPT CONDITIONING")
print("="*60)

CREATING DATASET WITH PROMPT CONDITIONING


In [ ]:
# ============================================================================
# CELL 6: Load Models & Setup LoRA
# ============================================================================
from diffusers import (
    StableDiffusionPipeline,
    UNet2DConditionModel,
    DDPMScheduler,
    AutoencoderKL
)
from transformers import CLIPTextModel, CLIPTokenizer
from peft import LoraConfig, get_peft_model

print("Loading Stable Diffusion 1.5 models...")

tokenizer = CLIPTokenizer.from_pretrained(
    config.pretrained_model_name,
    subfolder="tokenizer"
)

text_encoder = CLIPTextModel.from_pretrained(
    config.pretrained_model_name,
    subfolder="text_encoder"
)

vae = AutoencoderKL.from_pretrained(
    config.pretrained_model_name,
    subfolder="vae"
)

unet = UNet2DConditionModel.from_pretrained(
    config.pretrained_model_name,
    subfolder="unet"
)

noise_scheduler = DDPMScheduler.from_pretrained(
    config.pretrained_model_name,
    subfolder="scheduler"
)

print("✓ Models loaded")

# Freeze VAE and text encoder
vae.requires_grad_(False)
text_encoder.requires_grad_(False)
unet.requires_grad_(False)

# Enable gradient checkpointing for memory efficiency
unet.enable_gradient_checkpointing()

# Setup LoRA on UNet
lora_config = LoraConfig(
    r=config.lora_rank,
    lora_alpha=config.lora_alpha,
    init_lora_weights="gaussian",
    target_modules=["to_k", "to_q", "to_v", "to_out.0"],
    lora_dropout=config.lora_dropout,
)

unet = get_peft_model(unet, lora_config)
unet.print_trainable_parameters()

# Move models to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae.to(device)
vae.enable_slicing()
vae.enable_tiling()

print("✓ VAE memory optimizations enabled")
text_encoder.to(device)
unet.to(device)

print(f"✓ Models moved to {device}")

dataset = UnderwaterDataset(
    config.dataset_dir,
    config.repeat_weights,
    config.resolution,
    tokenizer=tokenizer
)

print(f"\n✓ Dataset created successfully!")

Loading Stable Diffusion 1.5 models...
✓ Models loaded
trainable params: 6,377,472 || all params: 865,898,436 || trainable%: 0.7365
✓ VAE memory optimizations enabled
✓ Models moved to cuda
  shutterstock-augmented: 531 images × 8 repeats = 4248 effective
  my-tank-underwater-scenes-augmented: 848 images × 1 repeats = 848 effective
  unsplash-augmented: 260 images × 1 repeats = 260 effective

✓ Dataset ready: 5356 total training samples with prompts

✓ Dataset created successfully!


In [ ]:
# ============================================================================
# CELL 7: Setup DataLoader and Optimizer
# ============================================================================
from torch.optim import AdamW
from diffusers.optimization import get_scheduler
from diffusers.models.autoencoders import AutoencoderKL

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    input_ids = torch.stack([example["input_ids"] for example in examples])
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids
    }

# Create dataloader
train_dataloader = DataLoader(
    dataset,
    batch_size=config.batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True
)

print(f"✓ DataLoader created: {len(train_dataloader)} batches per epoch")

# Setup optimizer
optimizer = AdamW(
    unet.parameters(),
    lr=config.learning_rate,
    betas=(0.9, 0.999),
    weight_decay=1e-2,
    eps=1e-8,
)

# Setup learning rate scheduler
num_training_steps = config.num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    config.lr_scheduler,
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=num_training_steps,
)

print(f"✓ Optimizer configured (lr={config.learning_rate})")
print(f"✓ Scheduler: {config.lr_scheduler} with {config.lr_warmup_steps} warmup steps")
print(f"  Total training steps: {num_training_steps}")

✓ DataLoader created: 670 batches per epoch
✓ Optimizer configured (lr=8e-05)
✓ Scheduler: cosine with 100 warmup steps
  Total training steps: 26800


In [ ]:
# ============================================================================
# CELL 8: Create Output Directories
# ============================================================================
os.makedirs(config.output_dir, exist_ok=True)
os.makedirs(f"{config.output_dir}/checkpoints", exist_ok=True)
os.makedirs(f"{config.output_dir}/validation_images", exist_ok=True)

print(f"✓ Output directories created at {config.output_dir}")

✓ Output directories created at /content/underwater_lora_output


In [ ]:
# ============================================================================
# CELL 9: Training Loop with Periodic GDrive Upload
# ============================================================================
from accelerate import Accelerator

def upload_best_model_to_gdrive(epoch, best_model_dir, max_retries=3):
    """Upload best LoRA weights to Google Drive with retry logic"""
    for attempt in range(max_retries):
        try:
            print(f"\n📤 Uploading best model to Google Drive (attempt {attempt+1}/{max_retries})...")

            if not os.path.exists(best_model_dir):
                print("⚠️ Best model not found, skipping upload")
                return False

            # Find the adapter file
            adapter_file = os.path.join(best_model_dir, "adapter_model.safetensors")
            if not os.path.exists(adapter_file):
                print(f"⚠️ adapter_model.safetensors not found in {best_model_dir}")
                return False

            # Get actual epoch number
            try:
                info_path = f"{config.output_dir}/checkpoints/lora_best_info.pth"
                if os.path.exists(info_path):
                    checkpoint = torch.load(info_path, map_location='cpu')
                    actual_epoch = checkpoint.get('epoch', epoch)
                else:
                    actual_epoch = epoch
            except:
                actual_epoch = epoch

            # Create timestamped filename
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            remote_filename = f'underwater_lora_best_epoch_{actual_epoch}_{timestamp}.safetensors'

            file_metadata = {
                'name': remote_filename,
                'parents': [GDRIVE_RESULTS_FOLDER_ID]
            }

            # Upload
            print(f"   Uploading: {os.path.basename(adapter_file)}")
            with open(adapter_file, 'rb') as f:
                media = MediaIoBaseUpload(
                    io.BytesIO(f.read()),
                    mimetype='application/octet-stream',
                    resumable=True
                )

            drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()

            print(f"✅ Best model uploaded: {remote_filename}")
            return True

        except Exception as e:
            print(f"⚠️ Upload attempt {attempt+1} failed: {e}")
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                print(f"   Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"❌ Upload failed after {max_retries} attempts")
                return False

# Setup accelerator
accelerator = Accelerator(
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    mixed_precision=config.mixed_precision,
)

# Prepare for training (including unet)
unet, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    unet, optimizer, train_dataloader, lr_scheduler
)

# Training state
global_step = 0
best_loss = float('inf')
best_model_path = f"{config.output_dir}/checkpoints/lora_best"
last_uploaded_epoch = 0

print("="*60)
print("STARTING TRAINING")
print("="*60)

for epoch in range(config.num_epochs):
    unet.train()
    epoch_loss = 0.0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{config.num_epochs}")

    for step, batch in enumerate(progress_bar):
        with accelerator.accumulate(unet):
            # Move batch to device with correct dtype
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)

            # Encode images to latent space
            with torch.no_grad():
                latents = vae.encode(pixel_values).latent_dist.sample()
                latents = latents * vae.config.scaling_factor

            # Sample noise
            noise = torch.randn_like(latents)
            bsz = latents.shape[0]

            # Sample random timesteps
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps,
                (bsz,), device=device
            ).long()

            # Add noise to latents
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Get text embeddings
            with torch.no_grad():
                encoder_hidden_states = text_encoder(input_ids)[0]

            # Predict noise
            model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

            # Calculate loss
            loss = F.mse_loss(model_pred.float(), noise.float(), reduction="mean")

            # Backpropagation
            accelerator.backward(loss)

            if accelerator.sync_gradients:
                accelerator.clip_grad_norm_(unet.parameters(), 1.0)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        # Logging
        if accelerator.sync_gradients:
            global_step += 1
            epoch_loss += loss.detach().item()

            progress_bar.set_postfix({
                "loss": f"{loss.detach().item():.4f}",
                "lr": f"{lr_scheduler.get_last_lr()[0]:.6f}"
            })

    # End of epoch
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    print(f"\nEpoch {epoch+1}/{config.num_epochs} - Average Loss: {avg_epoch_loss:.4f}")

    # Save best model
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        unwrapped_unet = accelerator.unwrap_model(unet)
        unwrapped_unet.save_pretrained(
            best_model_path,
            safe_serialization=True
        )
        # Save epoch info
        torch.save({
            'epoch': epoch + 1,
            'loss': avg_epoch_loss,
        }, f"{config.output_dir}/checkpoints/lora_best_info.pth")
        print(f"   🌟 New best model saved! Loss: {avg_epoch_loss:.4f}")

    # Upload to Google Drive
    if (epoch + 1) >= config.upload_after_epoch and \
        (epoch + 1 - last_uploaded_epoch) >= config.upload_interval_epochs:
        if upload_best_model_to_gdrive(epoch + 1, best_model_path):
            last_uploaded_epoch = epoch + 1

    # Save checkpoint periodically
    if (epoch + 1) % config.save_every_n_epochs == 0:
        checkpoint_path = f"{config.output_dir}/checkpoints/lora_epoch_{epoch+1}"
        accelerator.unwrap_model(unet).save_pretrained(
            checkpoint_path,
            safe_serialization=True
        )
        print(f"✓ Checkpoint saved: epoch {epoch+1}")

    # Validation
    #if (epoch + 1) % config.validation_every_n_epochs == 0:
    unet.eval()
    with torch.no_grad():
        # Create pipeline for generation
        pipeline = StableDiffusionPipeline.from_pretrained(
            config.pretrained_model_name,
            unet=accelerator.unwrap_model(unet),
            text_encoder=text_encoder,
            vae=vae,
            tokenizer=tokenizer,
            safety_checker=None,
            torch_dtype=torch.float16,
        )
        pipeline = pipeline.to(device)

        # Generate validation images (batch all prompts at once)
        images = pipeline(
            config.validation_prompts,
            num_inference_steps=30,
            guidance_scale=7.5,
        ).images

        # Save samples
        for i, img in enumerate(images):
            save_path = f"{config.output_dir}/validation_images/epoch_{epoch+1:03d}_prompt_{i+1}.png"
            img.save(save_path)

        print(f"✓ Generated {len(images)} validation images")

        del pipeline  # Free memory
    unet.train()

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)
print(f"Best loss: {best_loss:.4f}")
print(f"Best model saved at: {best_model_path}")

# Final model upload
print("\n📤 Uploading final best model...")
upload_best_model_to_gdrive(config.num_epochs, best_model_path)

print("\n✅ All done! Check your Google Drive for uploaded models.")

STARTING TRAINING


Epoch 1/40:   0%|          | 0/670 [00:00<?, ?it/s]


Epoch 1/40 - Average Loss: 0.1489
   🌟 New best model saved! Loss: 0.1489


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Expected types for unet: (<class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>,), got <class 'peft.peft_model.PeftModel'>.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


...

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Expected types for unet: (<class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>,), got <class 'peft.peft_model.PeftModel'>.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/30 [00:00<?, ?it/s]

✓ Generated 5 validation images


Epoch 36/40:   0%|          | 0/670 [00:00<?, ?it/s]

In [ ]:
# ============================================================================
# CELL 10: Final Model Upload
# ============================================================================
print("\n📤 Uploading final best model...")
upload_best_model_to_gdrive(config.num_epochs, best_model_path)

print("\n✅ All done! Check your Google Drive for uploaded models.")

In [ ]:
# Cell - AGGRESSIVE MEMORY CLEANUP
import gc
import torch
import os

# Move everything to CPU first
try:
    if 'unet' in globals():
        unet.cpu()
    if 'vae' in globals():
        vae.cpu()
    if 'text_encoder' in globals():
        text_encoder.cpu()
except:
    pass

# Delete everything
vars_to_delete = ['unet', 'vae', 'text_encoder', 'tokenizer', 'noise_scheduler',
                  'optimizer', 'train_dataloader', 'lr_scheduler', 'accelerator',
                  'pipeline', 'lora_config', 'batch', 'latents', 'noise']

for var in vars_to_delete:
    if var in globals():
        del globals()[var]

# Aggressive cleanup
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

# Reset CUDA memory allocator
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()

# Check memory
print(f"GPU memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"GPU memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")